In [1]:
import io
import arcpy
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.widgets import MapView
from zipfile import ZipFile
import os
from osgeo import ogr
from arcgis.geometry import SpatialReference
import sys
from arcpy.sa import *
import arcpy

from arcpy import env

In [2]:
from arcgis import GIS
gis = GIS()

In [3]:
# I created a points layer called "Dory" with her start and end point

In [4]:
#access landcover dataset via api
landcover_link = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/edu_umn/base_landcover_minnesota/tif_base_landcover_minnesota.zip'

mngeo_response = requests.get(landcover_link, allow_redirects=True)

In [5]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\tif_base_landcover_minnesota.zip", 'wb').write(mngeo_response.content)

97106583

In [6]:
#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\tif_base_landcover_minnesota.zip", 'r') as z:
    z.printdir()
    
    print('Extracting all files now..')
    z.extractall(r"C:\Users\bende287\Downloads\tif_base_landcover_minnesota")
    print('Done!')

File Name                                             Modified             Size
landcover_impervious_statewide2013_v2.tif      2016-07-21 11:21:44   1691043568
metadata/metadata.html                         2016-10-21 11:43:38        31278
metadata/metadata.xml                          2016-10-25 10:58:54        17434
metadata/preview.jpg                           2016-10-21 11:43:38       320230
metadata/Thumbs.db                             2016-11-22 13:50:36        22528
Extracting all files now..
Done!


In [7]:
#access dem dataset via api
dem_url = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
dem_response = requests.get(dem_url, allow_redirects=True)

In [8]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\fgdb_elev_30m_digital_elevation_model.zip", 'wb').write(dem_response.content)

402844618

In [9]:
#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\fgdb_elev_30m_digital_elevation_model.zip", 'r') as a:
    a.printdir()
    
    print('Extracting all files now..')
    a.extractall(r"C:\Users\bende287\Downloads\fgdb_elev_30m_digital_elevation_model")
    print('Done!')

File Name                                             Modified             Size
Digital Elevation Model - 30m Resolution.lyr   2011-06-14 11:56:48       140800
metadata/metadata.html                         2021-02-24 09:08:54        12251
metadata/metadata.xml                          2021-02-24 09:08:50         5308
metadata/old/metadata.html.2012_03_06_12_26_44 2012-03-06 12:26:44        28704
metadata/old/metadata.html.2012_03_06_12_28_21 2012-03-06 12:28:20        28704
metadata/old/metadata.html.2012_03_06_12_30_53 2012-03-06 12:30:52        28704
metadata/old/metadata.xml.2012_03_06_12_26_44  2012-03-06 12:26:44         7942
metadata/old/metadata.xml.2012_03_06_12_30_53  2012-03-06 12:30:52         7942
metadata/old/metadata.xml.2012_03_06_12_28_21  2012-03-06 12:28:20         7942
metadata/shdr_clrim3/shdr_clrim3.sgml          2010-06-09 18:00:26         7576
metadata/shdr_clrim3/shdr_clrim3.xml           2010-06-09 17:38:16         7600
metadata/shdr_clrim3/shdr_clrim3.html   

In [10]:
#identify slope from each raster cell
arcpy.ddd.Slope(
    in_raster=r"C:\Users\bende287\Downloads\fgdb_elev_30m_digital_elevation_model\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m\Band_1",
    out_raster=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\Slope_digita1",
    output_measurement="DEGREE",
    z_factor=1,
    method="PLANAR",
    z_unit="METER",
    analysis_target_device="GPU_THEN_CPU"
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\MyProject1\\MyProject1.gdb\\Slope_digita1'>

In [11]:
#clip slope layer
arcpy.management.Clip(
    in_raster="Slope_digita1",
    rectangle="558788.5 4862723.5 596543.5 4900928.5",
    out_raster=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\Slope_digita1_Clip1",
    in_template_dataset="landcover_impervious_st_Clip1",
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\MyProject1\\MyProject1.gdb\\Slope_digita1_Clip1'>

In [12]:
#reclassify slope layer on a 1-4 scale
arcpy.ddd.Reclassify(
    in_raster="Slope_digita1",
    reclass_field="VALUE",
    remap="0 8.53 1;8.53 16.700000 2;16.7 30.960000 3;30.96 90 4",
    out_raster=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\Reclass_Slope",
    missing_values="DATA"
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\MyProject1\\MyProject1.gdb\\Reclass_Slope'>

In [13]:
#reclassify landcover layer on same scale-- called "out_raster"
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="landcover_impervious_st_Clip1",
        reclass_field="Value",
        remap=" 0 101 1; 101 104 3; 104 108 2; 109 111 4",
        missing_values=0
    )
    out_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\Reclass_landcover1024")

In [14]:
#here i am giving more influence to the landcover layer
weightedlandcover = Raster("out_raster")*0.66

In [15]:
weightedslope = Raster("Reclass_Slope")*0.34

In [16]:
#using map algebra to combine our weighted datasets and create a cost surface to input into optimal region connections
combined_datasets = Raster("weightedslope") + Raster("weightedlandcover")

In [17]:
#this will create a line output connecting our two points, the route will be based on the cost surface
arcpy.sa.OptimalRegionConnections(
    in_regions="Dory",
    out_feature_class=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\Optimal_Dory1",
    in_barrier_data=None,
    in_cost_raster="combined_datasets",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

<geoprocessing server result object object at 0x000001D2A6C37240>

In [21]:
#testing out more weighting schemes for the uncertainty analysis
#still giving more importance to land cover layer here 
weightedlandcover2 = Raster("out_raster")*0.85

weightedslope2 = Raster("Reclass_Slope")*0.15

#using map algebra to combine our weighted datasets and create a cost surface to input into the optimal region connections function
combined_dataset2 = Raster("weightedslope2") + Raster("weightedlandcover2")

arcpy.sa.OptimalRegionConnections(
    in_regions="Dory",
    out_feature_class=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\Optimal_Dory2",
    in_barrier_data=None,
    in_cost_raster="combined_dataset2",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

<geoprocessing server result object object at 0x000001D2BC1B57A0>

In [23]:
#testing out more weighting schemes for the uncertainty analysis-- this time giving more importance to slope layer

weightedlandcover3 = Raster("out_raster")*0.40

weightedslope3 = Raster("Reclass_Slope")*0.60

#using map algebra to combine our weighted datasets and create a cost surface to input into the optimal region connections function
combined_dataset3 = Raster("weightedslope3") + Raster("weightedlandcover3")

arcpy.sa.OptimalRegionConnections(
    in_regions="Dory",
    out_feature_class=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\Optimal_Dory3",
    in_barrier_data=None,
    in_cost_raster="combined_dataset3",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

<geoprocessing server result object object at 0x000001D2BB2BDA10>